# What is the possibility for life on Exoplanets currently observed and those in the future?

## By: Ethan Barr and Tim Freerksen

# Introduction

Is there life in space? This question has been asked for years however no real signs of life have been observed yet despite the numerous exoplanets that we have discovered over the years. The objective of this project is to go through data regarding the exoplanets and their host stars.

In [11]:
# All Imports
import pandas as pd

In [12]:
data = pd.read_csv('PS.csv')
data.set_index('loc_rowid', inplace=True)
print(data.head())

            pl_name hostname  default_flag  sy_snum  sy_pnum  discoverymethod  \
loc_rowid                                                                       
1          11 Com b   11 Com             0        2        1  Radial Velocity   
2          11 Com b   11 Com             1        2        1  Radial Velocity   
3          11 UMi b   11 UMi             0        1        1  Radial Velocity   
4          11 UMi b   11 UMi             0        1        1  Radial Velocity   
5          11 UMi b   11 UMi             1        1        1  Radial Velocity   

           disc_year                           disc_facility  \
loc_rowid                                                      
1               2007                        Xinglong Station   
2               2007                        Xinglong Station   
3               2009  Thueringer Landessternwarte Tautenburg   
4               2009  Thueringer Landessternwarte Tautenburg   
5               2009  Thueringer Landessternwart